###  Created by Luis Alejandro (alejand@umich.edu)
Extracts TMID info from all signals into DB. **Careful!!! This will overwrite all entries in DB!!!**

In [ ]:
import mysql.connector
from nptdms import TdmsFile
import numpy as np
from scipy.signal import spectrogram
from scipy.signal import decimate 

In [ ]:
fs = 51200
factor = 6
# Connects to DB
mydb = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "cuba",
        database = "airnoise"
    )
# Gets signals
dbcursor = mydb.cursor()
sql = '''SELECT t.measurement, t.array, t.microphone FROM
        tmid t ORDER BY t.measurement, t.array, t.microphone;'''
dbcursor.execute(sql)
signals = dbcursor.fetchall()

In [ ]:
curr_measurement = None
for measurement,array,microphone in signals:
    
    if measurement != curr_measurement:
        dbcursor = mydb.cursor()        
        sql = '''SELECT m.url FROM measurements m WHERE
                    m.id_measurement = {:}'''.format(measurement)
        dbcursor.execute(sql)
        results = dbcursor.fetchall()
        url = results[0][0]
        tdms = TdmsFile("../../datasets/classification/noise/" + url)
        curr_measurement = measurement
    data = tdms.channel_data(group = 'Untitled', channel = 'cDAQ1Mod' + str(array) + '/ai' + str(microphone - 1))
    data = data/max(np.absolute(data))
    data = decimate(data,factor)
    f, t, Sxx = spectrogram(data,int(fs/factor),mode='magnitude',
                                window='hamming',nperseg=512,noverlap=256)
    power = Sxx.mean(axis=0)
    location = t[power.argmax()]
    dbcursor = mydb.cursor()
    sql = '''UPDATE tmid t SET t.location = {} WHERE t.measurement = {}
            AND t.array = {} AND t.microphone = {}'''.format(
            location,measurement,array,microphone)
    dbcursor.execute(sql)
    mydb.commit()
    print('Updating measurement: {}, array: {}, microphone: {} with location {}'.format(measurement,array,microphone,location))
    